1. Presentar IPython

2. Agregar librerías estandar

3. Agregar datasets con read_csv, desde archivos locales y url

4. Mostrar head, tail, index, columns, values, describe, T (transpose),

5. Problema con unicode (utilizar unidecode)

6. Explorar Pandas

7. Normalizar encabezados

8. Limpiar N/R con na_values en read_csv

!!! Checar cómo agregar la suma acumulativa a la última row

8. Suma acumulativa del PREP y Final, compararlos con una gráfica de barras

9. Que los votos en las actas contabilizadas del PREP sean iguales que en el Final

- Encontrar actas contabilizadas en prep
- Restar el total de votos en el final y en el prep
- Checar cuales partidos fueron afectados
- Plottear los datos (MISSING)

10. Checar las actas no contabilizadas. Comparar PREP y Final.

- Encontrar actas no contabilizadas en prep
- Mostrar el total de votos en el final con los índices de las actas no contabilizadas
- Checar cuales partidos fueron afectados
- Plottear los datos (MISSING)

NR que no registró candidato: http://v1ceeactas1.blob.core.windows.net/actas/A000101.JPG

In [33]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unidecode import unidecode

In [135]:
%ls data/

prep_ayuntamiento.csv  prep_diputados_locales.csv  prep_gobernador.csv


In [77]:
df_prep_ayuntamiento = pd.read_csv('data/prep_ayuntamiento.csv')
df_prep_diputados_loc = pd.read_csv('data/prep_diputados_locales.csv')
df_prep_gobernador = pd.read_csv('data/prep_gobernador.csv')

df_final_gobernador = pd.read_csv('http://ceesiprerepl3.cloudapp.net/sipre/gobernador.csv')

dataframes = [df_prep_ayuntamiento, df_prep_diputados_loc, df_prep_gobernador, df_final_gobernador]

*El PREP es un mecanismo de monitoreo censal, en tiempo real, del flujo de datos electorales de las casillas conforme llegan a los centros de acopio en las 300 juntas distritales del país. No es el resultado definitivo de la votación ni sustituye a los Cómputos Distritales.*

In [54]:
df_prep_gobernador.head()

,seccion,casilla,eleccion,municipio,distrito,estatus,ln,total,partido_accion_nacional,partido_revolucionario_institucional,...,combinacion_pri_pvem_pd,combinacion_pri_na_pd,combinacion_pvem_na_pd,combinacion_pri_pvem,combinacion_pri_na,combinacion_pri_pd,combinacion_pvem_na,combinacion_pvem_pd,combinacion_na_pd,votos_anulados
0,1,BASICA,Gobernador,Abasolo,20,Acta contabilizada,410,336,47,87,...,0,0,0,0,0,0,0,0,0,5
1,1,CONTIGUA 1,Gobernador,Abasolo,20,Acta contabilizada,410,335,24,127,...,0,0,0,0,0,0,0,1,0,8
2,2,BASICA,Gobernador,Abasolo,20,Acta contabilizada,695,555,65,181,...,0,0,0,0,1,0,0,0,0,16
3,2,CONTIGUA 1,Gobernador,Abasolo,20,Con incidencia: Presenta Correcciones,695,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,BASICA,Gobernador,Agualeguas,22,Acta contabilizada,419,325,133,99,...,0,0,0,2,0,0,0,0,0,4


In [78]:
for df in dataframes:
    for column in df.columns:
        df.rename(columns = { column : unidecode(column).lower().replace(' ', '_') }, inplace = True )

/home/mroutis/.local/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  app.launch_new_instance()


Verificar que las actas contabilizadas del PREP no hayan sufrido cambios en los resultados finales

In [149]:
prep_gob_actas_contabilizadas = df_prep_gobernador[ df_prep_gobernador.estatus == 'Acta contabilizada' ].index

df = DataFrame(df_final_gobernador.iloc[prep_gob_actas_contabilizadas].total - \
df_prep_gobernador.iloc[prep_gob_actas_contabilizadas].total)

incidencias = df[ df.total != 0 ].sort(ascending = True).index

In [150]:
df_final_gobernador.iloc[ incidencias ].head()

,seccion,casilla,eleccion,municipio,distrito,estatus,ln,total,partido_accion_nacional,partido_revolucionario_institucional,...,combinacion_pri_pvem_pd,combinacion_pri_na_pd,combinacion_pvem_na_pd,combinacion_pri_pvem,combinacion_pri_na,combinacion_pri_pd,combinacion_pvem_na,combinacion_pvem_pd,combinacion_na_pd,votos_anulados
12,10,BASICA,Gobernador,Agualeguas,22,Acta contabilizada,374,232,85,68,...,0,0,0,0,0,0,3,0,0,4
55,34,BASICA,Gobernador,Allende,24,Acta contabilizada,381,214,49,71,...,0,0,0,1,0,0,0,0,0,6
61,39,BASICA,Gobernador,An�huac,21,Acta contabilizada,750,520,157,160,...,0,0,0,1,0,0,0,0,0,17
70,43,BASICA,Gobernador,An�huac,21,Acta contabilizada,648,444,177,131,...,0,0,0,0,0,0,0,0,0,5
82,49,CONTIGUA 1,Gobernador,An�huac,21,Acta contabilizada,619,412,152,139,...,0,0,0,0,0,0,0,0,0,7


In [151]:
df_prep_gobernador.iloc[ incidencias ].head()

,seccion,casilla,eleccion,municipio,distrito,estatus,ln,total,partido_accion_nacional,partido_revolucionario_institucional,...,combinacion_pri_pvem_pd,combinacion_pri_na_pd,combinacion_pvem_na_pd,combinacion_pri_pvem,combinacion_pri_na,combinacion_pri_pd,combinacion_pvem_na,combinacion_pvem_pd,combinacion_na_pd,votos_anulados
12,10,BASICA,Gobernador,Agualeguas,22,Acta contabilizada,374,228,85,68,...,0,0,0,0,0,0,3,0,0,0
55,34,BASICA,Gobernador,Allende,24,Acta contabilizada,381,208,49,71,...,0,0,0,1,0,0,0,0,0,0
61,39,BASICA,Gobernador,An�huac,21,Acta contabilizada,750,514,151,160,...,0,0,0,1,0,0,0,0,0,17
70,43,BASICA,Gobernador,An�huac,21,Acta contabilizada,648,445,177,131,...,0,0,0,0,1,0,0,0,0,5
82,49,CONTIGUA 1,Gobernador,An�huac,21,Acta contabilizada,619,410,152,139,...,0,0,0,0,0,0,0,0,0,7
